# Unidad 05

In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"

In [ ]:
foreach (var line in System.IO.File.ReadLines(".env"))
{
    var parts = line.Split('=');
    if (parts.Length == 2)
    {
        var key = parts[0].Trim();
        var value = parts[1].Trim();
        Environment.SetEnvironmentVariable(key, value);
    }
}

string connectionString = "Server=" + Environment.GetEnvironmentVariable("Server") + ";Database=" + Environment.GetEnvironmentVariable("Database") + ";User Id=" + Environment.GetEnvironmentVariable("User") + ";Password=" + Environment.GetEnvironmentVariable("Password") + ";Encrypt=True;TrustServerCertificate=True;Connection Timeout=30;";


In [ ]:
#!connect mssql --kernel-name Pampero @csharp:connectionString

## DDM - Dynamic Data Masking

In [ ]:
-- creamos schema para los datos
CREATE SCHEMA Data;
GO
-- creamos tabla con campos con DDM
CREATE TABLE Data.Membership(
MemberID    int IDENTITY(1,1) NOT NULL PRIMARY KEY CLUSTERED,
FirstName       varchar(100) MASKED WITH (FUNCTION = 'partial(1, "xxxxx", 1)') NULL,
LastName        varchar(100) NOT NULL,
Phone          varchar(12) MASKED WITH (FUNCTION = 'default()') NULL,
Email          varchar(100) MASKED WITH (FUNCTION = 'email()') NOT NULL,
DiscountCode    smallint MASKED WITH (FUNCTION = 'random(1, 100)') NULL
);

In [ ]:
-- insertamos datos de ejemplo
INSERT INTO Data.Membership (FirstName, LastName, Phone, Email, DiscountCode)
VALUES
('Roberto', 'Tamburello', '555.123.4567', 'RTamburello@contoso.com', 10),
('Janice', 'Galvin', '555.123.4568', 'JGalvin@contoso.com.co', 5),
('Shakti', 'Menon', '555.123.4570', 'SMenon@contoso.net', 50),
('Zheng', 'Mu', '555.123.4569', 'ZMu@contoso.net', 40);

In [ ]:
CREATE USER MaskingTestUser WITHOUT LOGIN; 
GRANT SELECT ON SCHEMA::Data TO MaskingTestUser; 

-- ejecutamos la consulta como el usuario para probar:
EXECUTE AS USER = 'MaskingTestUser'; 
SELECT * FROM Data.Membership; 
REVERT;

In [ ]:
GRANT UNMASK TO MaskingTestUser; 
EXECUTE AS USER = 'MaskingTestUser'; 
SELECT * FROM Data.Membership; 
REVERT;

-- Quitar permiso de UNMASK
REVOKE UNMASK TO MaskingTestUser;